## Setup

In [140]:
from dotenv import load_dotenv
from utils import chat_interface
import os
import math

from langgraph.checkpoint.memory import MemorySaver
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.tools import tool
from langgraph.graph.message import MessagesState
from langchain_core.prompts import PromptTemplate
from langchain_chroma import Chroma
from langchain_core.documents import Document
from pydantic import BaseModel, Field, ConfigDict
from typing import Dict, Any, List, Annotated, Literal, Tuple
from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

### Parameters

In [27]:
num_retrieval = 5

### Setup LLMs for different Technical Support Level

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# llm_base_url = "https://openai.vocareum.com/v1"
llm_base_url = "https://api.openai.com/v1"


llm_low = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.0,
    base_url=llm_base_url,
    api_key=OPENAI_API_KEY,
)

llm_medium = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
    base_url=llm_base_url,
    api_key=OPENAI_API_KEY,
)

llm_high = ChatOpenAI(
    model="gpt-4o",
    temperature=0.0,
    base_url=llm_base_url,
    api_key=OPENAI_API_KEY,
)

embeddings_fn = OpenAIEmbeddings(
    model="text-embedding-3-large",
    base_url=llm_base_url,
    api_key=OPENAI_API_KEY,
)

### Create Vector Database Connection for RAG

In [29]:
chromadb_directory = "vectorstore"
collection_name = "knowledge_vecotr_store"
vector_store = Chroma(
    collection_name=collection_name,
    embedding_function=embeddings_fn,
    persist_directory=chromadb_directory,
)

## Agent State Management

State Memory

In [ ]:
class TicketState(MessagesState):
    """State for the IT ticket workflow."""
    query: HumanMessage
    # retrieved documents
    documents: List[Document]
    # routing signals
    priority: Literal['normal', 'urgent', 'vip'] = 'normal'
    intent: Literal['how_to', 'troubleshooting_basic', 'status_check',
                    'security', 'refund_high', 'outage', 'others'] = 'others'
    retrieval_score: float = 0.0
    answer_confidence: float = 0.0
    sentiment: float = 0.0
    # control/status
    assigned_group: Literal['L1', 'L2', 'L3', 'human'] = 'L1'
    eval_status: Literal['resolved', 'needs_clarification', 'escalate'] = 'needs_clarification'
    turns_at_level: int = 0
    escalation_reason: Optional[Literal['low_confidence',
                                        'high_impact_or_sensitive',
                                        'unresolved_or_negative',
                                        'sla_breach']] = None
    # SLA/aging
    level_entered_at: datetime = datetime.utcnow()
    sla_minutes: int = 3  # example per level

Standardized Output for Subagent

In [147]:
class IntentOutput(BaseModel):
    """Standardized output to ensure it is from the possible values"""
    intent: Annotated[Literal['how_to','troubleshooting_basic','status_check','security','refund_high','outage','others'], 
                      Field(description="The intent of the IT ticket")]
    
class SentimentOutput(BaseModel):
    """Standardized output to ensure it is from the possible values"""
    sentiment: Annotated[float, Field(description="The user's sentiment",
                                      ge=-1.0,
                                      le=1.0)]

class ConfidenceOutput(BaseModel):
    """Standardized output to ensure it is from the possible values"""
    confidencet: Annotated[float, Field(description="The confidence of answer toward the query",
                                      ge=0.0,
                                      le=1.0)]

# ChromaDB Similarity Search Output Structure        
class VectorDocument(BaseModel):
    """The contents of a retrieved LangChain Document."""
    id: int
    page_content: str
    metadata: Dict[str, Any]

class SearchResultItem(BaseModel):
    """Represents the tuple (Document, score) as a JSON object."""
    document: VectorDocument = Field(description="The document content and metadata.")
    score: float = Field(description="The L2 distance.")

class QueryAttributeOutput(BaseModel):
    """Standardized output to ensure it is from the possible values"""
    intent: Annotated[Literal['how_to','troubleshooting_basic','status_check','security','refund_high','outage','others'], 
                      Field(description="The intent of the IT ticket")]
    sentiment: Annotated[float, Field(description="The user's sentiment",
                                      ge=-1.0,
                                      le=1.0)]
    confidence: Annotated[float, Field(description="The confidence of answer toward the query",
                                      ge=0.0,
                                      le=1.0)]
    retrival: Annotated[List[SearchResultItem], Field(description="List of retrieved document and score pairs.")]


# Agent Tools

In [136]:
@tool
def knowledge_retrival(query: str) -> List[dict]:
    """
    Evaluate whether the knowledge base has enough content to address the query and return the relevant content indices.

    Args:
        query (str): The user query string
    
    Returns:
        retrival (List[tuple[Document,float]]): Response from the ChromaDB database for the retrieved documents and the corresponding
                                                distance measure.
    """
    results = vector_store.similarity_search_with_score(
        query=query,
        k=num_retrieval,
    )

    # Convert the output to recognizable types for Pydantic to work
    retrival = []
    for doc, score in results:
        retrival.append({
            "document": {
                "id": int(doc.id),
                "page_content": doc.page_content,
                "metadata": doc.metadata
            },
            "score": score
        })
    return retrival

In [61]:
@tool
def intent_check(query: str) -> str:
    """
    Evaluate the intent of the query for routing decision.

    Args:
        query (str): The user query string
    
    Returns:
        intent (str): One of ['how_to', 'troubleshooting_basic', 'status_check',
                              'security', 'refund_high', 'outage', 'others']
    """
    
    prompt_template = """
        You are an expert evaluating the intent of IT ticket submitted by users and you are suppose to
        classify it as one of the following intent: ['how_to', 'troubleshooting_basic', 'status_check',
        'security', 'refund_high', 'outage', 'others'].

        Output a single one of the above intent in the list, do not provide any explanation or description
        to your answer.

        Here is the user query:
        {query}
        """
    prompt = PromptTemplate(
        template = prompt_template,
        input_variables = ["query"],
        ).invoke({"query": query})
    response = llm_medium.with_structured_output(IntentOutput).invoke(prompt)
    intent = response.intent
    return intent
    

In [62]:
@tool
def sentiment_analyzer(query: str) -> float:
    """
    Evaluate the sentiment of the input query and output a score between -1 to 1.

    Args:
        query (str): The user query string
    
    Returns:
        sentiment (float): Score between -1 to 1, where -1 is upset/angry/negative where 1 is happy/optimisitic/positive
    """
    prompt_template = """
        You are an expert in marketing and assessing the sentiment of the customer.  You need to provide a score between
        -1 to 1 where -1 is upset/angry/negative where 1 is happy/optimisitic/positive.

        Output a single number, do not provide any explanation or description to your answer.

        Here is the user query:
        {query}
        """
    prompt = PromptTemplate(
        template = prompt_template,
        input_variables = ["query"],
        ).invoke({"query": query})
    response = llm_medium.with_structured_output(SentimentOutput).invoke(prompt)
    sentiment = response.sentiment
    return sentiment

In [63]:
@tool
def confidence_evaluator(query: str, answer: str) -> float:
    """
    Evaluate the confidence of the answer towards answering the query by providing a score between 0 to 1.

    Args:
        query (str): The user query string
        answer (str): The answer provided by the IT support
    
    Returns:
        confidence (float): Score between 0 to 1, where 0 corresponse to an answer where it is completely irrelevant to
        the querey, where 1 is spot on and it solves the user's query.
    """
    prompt_template = """
        You are an expert in quality control to evaluate whether the support team had been doing a good job for providing
        the relevant response to the user.  You need to provide a score between 0 to 1, where 0 corresponse to an answer 
        where it is completely irrelevant to the querey, where 1 is spot on and it solves the user's query.

        Here is the user query:
        {query}

        Here is the support team's response:
        {answer}
        """
    prompt = PromptTemplate(
        template = prompt_template,
        input_variables = ["query","answer"],
        ).invoke({"query": query, "answer": answer})
    response = llm_medium.with_structured_output(ConfidenceOutput).invoke(prompt)
    confidence = response.confidence
    return confidence

In [ ]:
llm_initial_evaluator = llm_medium.bind_tools([knowledge_retrival,intent_check,sentiment_analyzer])

## Agents

In [ ]:
L1_TOPICS = {"how_to","troubleshooting_basic","status_check"}
L2_TOPICS = {"others","refund_high"}
L3_TOPICS = {"outage","security"}

In [ ]:
def query_assessment_agent(state: TicketState):
    query = state["query"]

    # Evaluate Retrival Score, Intent, Sentiment of the Query
    prompt_template = """
        You are a IT ticket evaluation agent, given the user query:
        {query}

        Use the following tools to provide insights for the user query:
            (1) Use `intent_check` to identify the "intent" attribute of the given query
            (2) Use `sentiment_analyzer` to estimate the "sentiment" of the given query
            (3) Use `knowledge_retrival` to extract "retrival" attribute from the ChromaDB knowledge database
        """
    prompt = PromptTemplate(
        template = prompt_template,
        input_variables = ["query"],
        ).invoke({"query": query})

    response = llm_medium.with_structured_output(QueryAttributeOutput).invoke(prompt)

    llm_initial_evaluator.invoke
    similarity = [ 1-(result[1]/math.sqrt(2)) for result in results ]  # ChromaDB uses L2 distance, it requires to perform this operation to convert to similarity
    max_score = max(similarity)
    retrival_score = max_score

    indices = [ int(result[0].id) for result in results ]

In [ ]:
# TODO: Develop your agents under `agentic/agents`
# TODO: Develop your tools under `agentic/tools`
# TODO: Modify `agentic/workflow` in order to orchestrate your agents


In [ ]:
# IDEALLY YOUR ONLY IMPORT HERE IS:
# from agentic.workflow import orchestrator

from agentic.workflow import orchestrator

In [ ]:
chat_interface(orchestrator, "1")

In [ ]:
list(orchestrator.get_state_history(
    config = {
        "configurable": {
            "thread_id": "1",
        }
    }
))[0].values["messages"]

---

# Testing Code

In [151]:
query = "Hi, I have a question regarding the billing for the events I attended last month. There seems to be a discrepancy in the charges, and I would like to understand the breakdown of the costs. Could you please provide a detailed billing statement? Thank you."
results = vector_store.similarity_search_with_score(
    query=query,
    k=num_retrieval,
)
# similarity = [ 1-(result[1]/math.sqrt(2)) for result in results ]
# indices = [ int(result[0].id) for result in results ]

retrival = []
for doc, score in results:
    retrival.append({
        "document": {
            "id": int(doc.id),
            "page_content": doc.page_content,
            "metadata": doc.metadata
        },
        "score": score
    })

In [154]:
class VectorDocument(BaseModel):
    """The contents of a retrieved LangChain Document."""
    id: int
    page_content: str
    metadata: Dict[str, Any]

class SearchResultItem(BaseModel):
    """Represents the tuple (Document, score) as a JSON object."""
    document: VectorDocument = Field(description="The document content and metadata.")
    score: float = Field(description="The L2 distance.")

class QueryAttributeOutput(BaseModel):
    retrival: Annotated[List[SearchResultItem], Field(description="List of retrieved document and score pairs.")]

In [158]:
QueryAttributeOutput(retrival=retrival)

QueryAttributeOutput(retrival=[SearchResultItem(document=VectorDocument(id=19, page_content='How to Handle Billing Discrepancies: For billing issues:\n\n- Review your billing history in the CultPass app\n- Contact support if discrepancies are found\n- Provide transaction details for faster resolution\n\n**Suggested phrasing:**\n"If you notice any billing discrepancies, review your history and contact support with transaction details."', metadata={'tags': 'billing, escalation, support'}), score=1.0406265258789062), SearchResultItem(document=VectorDocument(id=5, page_content='Understanding CultPass Pricing: CultPass offers flexible pricing options:\n\n- Monthly subscription: Access to 4 experiences per month\n- Annual subscription: Discounted rate for a year-long access\n- Additional fees may apply for premium events\n\n**Suggested phrasing:**\n"Explore our flexible pricing plans, including monthly and annual subscriptions. Note that some premium events may incur additional fees."', meta

In [153]:
query = "Hi, I have a question regarding the billing for the events I attended last month. There seems to be a discrepancy in the charges, and I would like to understand the breakdown of the costs. Could you please provide a detailed billing statement? Thank you."
prompt_template = """
    You are an expert evaluating the intent of IT ticket submitted by users and you are suppose to
    classify it as one of the following intent: ['how_to', 'troubleshooting_basic', 'status_check',
    'security', 'refund_high', 'outage', 'others'].

    Output a single one of the above intent in the list, do not provide any explanation or description
    to your answer.

    Here is the user query:
    {query}
    """
prompt = PromptTemplate(
    template = prompt_template,
    input_variables = ["query"],
    ).invoke({"query": query})
response = llm_medium.with_structured_output(IntentOutput).invoke(prompt)


In [54]:
query = "Hi, I have a question regarding the billing for the events I attended last month. There seems to be a discrepancy in the charges, and I would like to understand the breakdown of the costs. Could you please provide a detailed billing statement? Thank you."
query = "You are so useless!"
prompt_template = """
    You are an expert in marketing and assessing the sentiment of the customer.  You need to provide a score between
    -1 to 1 where -1 is upset/angry/negative where 1 is happy/optimisitic/positive.

    Output a single number, do not provide any explanation or description to your answer.

    Here is the user query:
    {query}
    """
prompt = PromptTemplate(
    template = prompt_template,
    input_variables = ["query"],
    ).invoke({"query": query})
response = llm_medium.with_structured_output(SentimentOutput).invoke(prompt)
sentiment = response.sentiment
sentiment

-1.0

In [156]:
query = "Hi, I have a question regarding the billing for the events I attended last month. There seems to be a discrepancy in the charges, and I would like to understand the breakdown of the costs. Could you please provide a detailed billing statement? Thank you."

# Evaluate Retrival Score, Intent, Sentiment of the Query
prompt_template = """
    You are a IT ticket evaluation agent, given the user query:
    {query}

    Use `knowledge_retrival` to extract "retrival" attribute from the ChromaDB knowledge database
    """
prompt = PromptTemplate(
    template = prompt_template,
    input_variables = ["query"],
    ).invoke({"query": query})

response = llm_initial_evaluator.with_structured_output(QueryAttributeOutput).invoke(prompt)

BadRequestError: Error code: 400 - {'error': {'message': "Invalid schema for response_format 'QueryAttributeOutput': In context=('properties', 'document', 'properties', 'metadata'), 'additionalProperties' is required to be supplied and to be false.", 'type': 'invalid_request_error', 'param': 'response_format', 'code': None}}

In [77]:
response

QueryAttributeOutput(intent='refund_high', sentiment=0.5, confidencet=0.85)